<a href="https://colab.research.google.com/github/zxzxzx131313/S22_neural_bodies_StyleGan2/blob/main/020622_Exercise1_StyleGAN2_Colab_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Training StyleGAN2 on Colab
If it were me I’d sign up for Colab Pro ($10/month) to get a couple extra hours of training time in per session. The notebook is broken up into a couple of parts the first is training the network. Then testing it with a latent walk. Before we get into too many details of the process - here are the main steps of generating the walk. 
- Get the StyleGAN2 repo so that we can use the scripts from it
- Get the training dataset, this is what you scrape and stuff
- Then we point the model from the repo and train a model
- Lastly we generate images which are a walk in this latent space

##Mounting Google Drive
So connect up to your google drive so that you can download the repo directly into your drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Install the repo
**Only do this for the first time ever setting this up!**

If this is your first time ever running this notebook, you’ll want to install my fork of StyleGAN2 to your Drive account. Make sure you have ample space on your Drive (I’d say at least 50GB). This will install the repo and add some dependecies (like transferring from FFHQ the first time).

Every time after your first use of this notebook you’ll want to skip this cell and run the cell after this.

In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
# this makes a folder 
%cd /content/drive/My/Drive/
!mkdir stylegan2-colab
# then we get into the folder
%cd stylegan2-colab/
# this clones the repository 
!git clone https://github.com/dvschultz/stylegan2
# then we go into the repo
%cd stylegan2
# then we make a folder called "pkl"
!mkdir pkl
%cd pkl
# the gdown commands downloads a file and the ID is whats specified
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
# then make a results drirectory
!mkdir results
# and under results we would make and navigate to a folder called 00001-pretrained
!mkdir results/00001-pretrained
%cd results/00001-pretrained
# download that too! 
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
# lastly make a folder called datasets
%mkdir datasets

[Errno 2] No such file or directory: '/content/drive/My/Drive/'
/content/stylegan2-colab/stylegan2
/content/stylegan2-colab/stylegan2/stylegan2-colab
Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 32.64 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/stylegan2-colab/stylegan2/stylegan2-colab/stylegan2
/content/stylegan2-colab/stylegan2/stylegan2-colab/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/stylegan2-colab/stylegan2/stylegan2-colab/stylegan2/pkl/inception_v3_features.pkl
100% 87.3M/87.3M [00:00<00:00, 166MB/s]
/content/stylegan2-colab/stylegan2/stylegan2-colab/stylegan2
/content/stylegan2-colab/stylegan2/stylegan2-colab/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/stylegan2-colab/st

##Picking up from a previous session
If you already have the StyleGAN2 repo installed in Drive skip the above cell and run the following. This will make sure you have the latest version in case I make updates.

In [ ]:
#USE this if you already have a stylegan2 folder in google drive
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!git pull

[Errno 2] No such file or directory: '/content/drive/My Drive/stylegan2-colab/stylegan2'
/content/stylegan2-colab/stylegan2/stylegan2-colab/stylegan2
Already up to date.


##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [ ]:
%tensorflow_version 1.x

##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). 

I’ve seen some recommendations to run this command every time you restart your Colab machine. I think if you ahve a small-ish dataset (< 2000 images) that’s probably unnecessary.

I recommend you upload your dataset to Google Drive and copy its path from the Drive folder in Colab and paste its path in the below cell.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [ ]:
import os # this gives a nifty thing to list out directories 
os.listdir('/content/drive/MyDrive/ChinatownGate_Combined') # just make sure the directoy is right

FileNotFoundError: ignored

In [ ]:
%cd /content/drive/MyDrive/stylegan2-colab/stylegan2
%pwd

/content/drive/MyDrive/stylegan2-colab/stylegan2


'/content/drive/MyDrive/stylegan2-colab/stylegan2'

In [ ]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
#so an argument looks like this 
#python dataset_tool.py (the python file) create_from_images (the function) ./datasets/hybrid-512 (the place to dump the TF files) 
!python dataset_tool.py create_from_images '/content/drive/MyDrive/stylegan2-colab/stylegan2/datasets/NB_test1' '/content/drive/MyDrive/Neural Bodies/Datasets/TA_Meeting_New_Data_set' #Drive/beauxart/01-data/210122-plansectelev-512-stylegan

Loading images from "/content/drive/MyDrive/Neural Bodies/Datasets/TA_Meeting_New_Data_set"
Error: Input image resolution must be a power-of-two


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.

`--metrics`

METRICS DON’T MATTER. It’s art! Use your eyes. Set `--metrics=None` and live your life.

If you must use metrics, you have a few options. `fid50k`, the default, uses Frechet Inception Distance score. It’s what was used in StyleGAN1 and what most people know. It’s fine for images of animals and things, but it’s not great. `ppl_wend` is what StyleGAN2 prefers and claims to be more accurate. There are a bunch of other options but I’d recommend you stick with those. Note that both of these take 30–45minutes to run every time it runs so that cuts into your training time in Colab.

In [ ]:
!pip install numpy==1.19.5 # since we are using an older version of tensorflow -- if the cell after this ones fails run this one.

In [ ]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=hybrid-512 --mirror-augment=false --metrics=None

Local submit - run_dir: results/00002-stylegan2-hybrid-512-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Traceback (most recent call last):
  File "run_training.py", line 202, in <module>
    main()
  File "run_training.py", line 197, in main
    run(**vars(args))
  File "run_training.py", line 128, in run
    dnnlib.submit_run(**kwargs)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/submit.py", line 343, in submit_run
    return farm.submit(submit_config, host_run_dir)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/internal/local.py", line 22, in submit
    return run_wrapper(submit_config)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/submit.py", line 280, in run_wrapper
    run_func_obj(**submit_config.run_func_kwargs)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/training/training_loop.py", li

Once running, your training files will show up in the results folder.

#Testing the model (generating images)
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [ ]:
!pip install opensimplex
import opensimplex

     |████████████████████████████████| 15.7 MB 28.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.4.0 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.21.5 which is incompatible.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatibl

In [ ]:
import os 
os.listdir('/content/drive/MyDrive/stylegan2-colab/stylegan2/results/00001-pretrained') #network-snapshot-10000.pkl

['network-snapshot-10000.pkl']

In [ ]:
!python run_generator.py generate-images --network=/content/drive/MyDrive/stylegan2-colab/stylegan2/results/00001-pretrained/network-snapshot-10000.pkl --seeds=0-1000 --truncation-psi=0.5

python3: can't open file 'run_generator.py': [Errno 2] No such file or directory


Let’s zip the generated files and download them.

In [ ]:
# check the folder which has the images in it  - in this case it is 00004-generate-images
!zip -r generated-0.7.zip /content/drive/MyDrive/stylegan2-colab/stylegan2/results/00004-generate-images

	zip warning: name not matched: /content/drive/MyDrive/stylegan2-colab/stylegan2/results/00004-generate-images

zip error: Nothing to do! (try: zip -r generated-0.7.zip . -i /content/drive/MyDrive/stylegan2-colab/stylegan2/results/00004-generate-images)


##Interpolation


In [ ]:
!python run_generator.py generate-latent-walk --network=/content/drive/MyDrive/stylegan2-colab/stylegan2/results/00001-pretrained/network-snapshot-10000.pkl --seeds=0,3,10,13,17,38,42,43,46,48,55,63,67,69,85,88,92,97,98,99,117,119,124,137,139,141,146,152,156,160,173,177,180,189,191,195,198,203,205,212,220,221,228,230,247,267,271,290,292,293,297,300,301,302,306,309,316,318,319,321,324,332,338,340,364,365,371,393,396,427,435,450,458,459,465,468,469,471,487,493,497,498,499,505,516,521,523,525,542,543,548,573,576,577,588,593,597,621,622,627,632,637,657,694,695,704,705,707,712,713,724,726,737,739,752,761,771,779,780,789,797,0	 --frames 7192 --truncation-psi=0.5

python3: can't open file 'run_generator.py': [Errno 2] No such file or directory


In [ ]:
#convert to video 
!ffmpeg -r 24 -i /content/drive/MyDrive/stylegan2-colab/stylegan2/results/00005-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-sectelev.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# this is to remove the folder
#rm -r /content/drive/My Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f

rm: cannot remove '/content/drive/My': No such file or directory
rm: cannot remove 'Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f': No such file or directory
